# 控制LLMs输出结构

由于训练数据、模型参数规模、模型能力限制，大部分开源模型比如Llama2, Mistral, Baichuan2等都无法像GPT-4一样很好的支持`function call`或者按照指定的格式输出要求返回结构。

基于对语言模型预测token概率的采样规则修改，比如在需要语言模型输出数值时，屏蔽非数值token的采样概率，达到让语言模型按照指定规范生成的效果。这是在开源语言模型能力不足的过渡方法，其中一些开源方案：
- [llamacpp grammar](https://github.com/ggerganov/llama.cpp/blob/master/grammars/README.md)，灵活度中等，劣势是只支持llamacpp后端 
- [guidance](https://github.com/guidance-ai/guidance)更加灵活，支持llamacpp/hf transformers后端，缺点是不稳定
- [localai](https://localai.io/features/openai-functions)是对llamacpp grammar 的封装
- [functionary](https://github.com/MeetKai/functionary)只支持很少的模型
- [https://github.com/1rgs/jsonformer](jsonformer)
- [lm-format-enforcer](https://github.com/noamgat/lm-format-enforcer)，活跃项目，支持vllm/llamacpp 
- [outlines]()

## 🚀如何基于llamacpp grammar控制模型稳定输出Json

llamacpp grammar基于一套特殊的语法，我们可以先使用`TypeScript`定义数据结构，并在[intrinsiclabs](https://grammar.intrinsiclabs.ai/)上自动转换成`grammar`语法。

比如我们期望语言模型输出类似于下结构的Json:
```json
{
  "game_state": string, # "game over" 或 "game on progress",
  "message": string,
  "active_player": string,
}
```

我们先定义一个TypeScript如下，其中
```typescript
interface DM {
  game_state: GameState;
  active_player: string;
  message: string;
}

enum GameState {
  GameOver = "game over",
  GameOnProgress = "game on progress",
}
```

进入[intrinsiclabs](https://grammar.intrinsiclabs.ai/)复制粘贴就生成了`grammar`：

```grammar
root ::= DM
GameState ::= "\"game over\"" | "\"game on progress\""
DM ::= "{"   ws   "\"game_state\":"   ws   GameState   ","   ws   "\"active_player\":"   ws   string   ","   ws   "\"message\":"   ws   string   "}"
DMlist ::= "[]" | "["   ws   DM   (","   ws   DM)*   "]"
string ::= "\""   ([^"]*)   "\""
boolean ::= "true" | "false"
ws ::= [ \t\n]*
number ::= [0-9]+   "."?   [0-9]*
stringlist ::= "["   ws   "]" | "["   ws   string   (","   ws   string)*   ws   "]"
numberlist ::= "["   ws   "]" | "["   ws   string   (","   ws   number)*   ws   "]"

```

然后我们在模型预测的时候，传入`grammar`，模型就会按照我们的要求生成Json格式的输出了。

In [8]:
%%writefile test_grammar
root ::= DM
GameState ::= "\"game over\"" | "\"game on progress\""
DM ::= "{"   ws   "\"game_state\":"   ws   GameState   ","   ws   "\"active_player\":"   ws   string   ","   ws   "\"message\":"   ws   string   "}"
DMlist ::= "[]" | "["   ws   DM   (","   ws   DM)*   "]"
string ::= "\""   ([^"]*)   "\""
boolean ::= "true" | "false"
ws ::= [ \t\n]*
number ::= [0-9]+   "."?   [0-9]*
stringlist ::= "["   ws   "]" | "["   ws   string   (","   ws   string)*   ws   "]"
numberlist ::= "["   ws   "]" | "["   ws   string   (","   ws   number)*   ws   "]"

Overwriting test_grammar


In [12]:
import json

In [11]:
from llama_cpp import Llama, LlamaGrammar
model = Llama("/data/hf/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf")

llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from /data/hf/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mixtral-8x7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:     

llama_model_loader: - kv  15:                      tokenizer.ggml.tokens arr[str,32000]   = ["<unk>", "<s>", "</s>", "<0x00>", "<...
llama_model_loader: - kv  16:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  17:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  18:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  19:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  20:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  21:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  22:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  23:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  24: 

In [31]:
# completion with grammar
grammar = LlamaGrammar.from_file("test_grammar")
prompt = """Player1 and Player2 are playing a game named happy ending, following is the conversation between the two players:

Player1: hi
Player2: hello, how do you do recently?

Now reponse the game state, action player and message with Json format. Message involves the environment description string."""
res = model.create_completion(prompt, max_tokens=1000, grammar=grammar)
print(json.loads(res["choices"][0]['text']))

from_string grammar:
root ::= DM 
DM ::= [{] ws ["] [g] [a] [m] [e] [_] [s] [t] [a] [t] [e] ["] [:] ws GameState [,] ws ["] [a] [c] [t] [i] [v] [e] [_] [p] [l] [a] [y] [e] [r] ["] [:] ws string [,] ws ["] [m] [e] [s] [s] [a] [g] [e] ["] [:] ws string [}] 
GameState ::= ["] [g] [a] [m] [e] [ ] [o] [v] [e] [r] ["] | ["] [g] [a] [m] [e] [ ] [o] [n] [ ] [p] [r] [o] [g] [r] [e] [s] [s] ["] 
ws ::= ws_11 
string ::= ["] string_8 ["] 
DMlist ::= [[] []] | [[] ws DM DMlist_7 []] 
DMlist_6 ::= [,] ws DM 
DMlist_7 ::= DMlist_6 DMlist_7 | 
string_8 ::= string_9 
string_9 ::= [^"] string_9 | 
boolean ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] 
ws_11 ::= [ <U+0009><U+000A>] ws_11 | 
number ::= number_13 number_14 number_15 
number_13 ::= [0-9] number_13 | [0-9] 
number_14 ::= [.] | 
number_15 ::= [0-9] number_15 | 
stringlist ::= [[] ws []] | [[] ws string stringlist_18 ws []] 
stringlist_17 ::= [,] ws string 
stringlist_18 ::= stringlist_17 stringlist_18 | 
numberlist ::= [[] ws []] | [[] ws string

{'game_state': 'game over', 'active_player': 'Player2', 'message': 'This is a happy ending after Player1 says hi and Player2 replies hello.'}


In [17]:
# completion without grammar
res = model.create_completion(prompt, max_tokens=1000)
print(json.loads(res["choices"][0]['text']))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2468.43 ms
llama_print_timings:      sample time =     106.04 ms /   291 runs   (    0.36 ms per token,  2744.20 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   44112.52 ms /   291 runs   (  151.59 ms per token,     6.60 tokens per second)
llama_print_timings:       total time =   44874.16 ms /   292 tokens


{'gameState': 'ongoing', 'actionPlayer': 'Player2', 'message': {'conversation': {'current': [{'player': 'Player1', 'statement': 'hi'}, {'player': 'Player2', 'statement': 'hello, how do you do recently?'}]}, 'environment': {'players': [{'id': 'Player1', 'name': 'Player1', 'status': 'active', 'score': 0}, {'id': 'Player2', 'name': 'Player2', 'status': 'active', 'score': 0}], 'gameName': 'happy ending'}}}


Compare to the regular completion, using grammar for prediction has excellent consequences.

## 😀 A more preferred way for Python!

In [21]:
from pydantic import BaseModel
from enum import Enum

class GameState(str, Enum):
    game_over = "game over"
    game_active = "game on progress"

class DM(BaseModel):
    game_state : GameState
    message: str
    active_player: str

grammer = LlamaGrammar.from_json_schema(DM.schema_json())

/tmp/ipykernel_77046/3400868137.py:13: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  grammer = LlamaGrammar.from_json_schema(DM.schema_json())
from_string grammar:
space ::= space_1 
space_1 ::= [ ] | 
game-state-GameState ::= ["] [g] [a] [m] [e] [ ] [o] [v] [e] [r] ["] | ["] [g] [a] [m] [e] [ ] [o] [n] [ ] [p] [r] [o] [g] [r] [e] [s] [s] ["] 
string ::= ["] string_6 ["] space 
string_4 ::= [^"\] | [\] string_5 
string_5 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_6 ::= string_4 string_6 | 
root ::= [{] space ["] [g] [a] [m] [e] [_] [s] [t] [a] [t] [e] ["] space [:] space game-state-GameState [,] space ["] [m] [e] [s] [s] [a] [g] [e] ["] space [:] space string [,] space ["] [a] [c] [t] [i] [v] [e] [_] [p] [l] [a] [y] [e] [r] ["] space [:] space string [}] s

In [22]:
res = model.create_completion(prompt, max_tokens=1000, grammar=grammar)
print(json.loads(res["choices"][0]['text']))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2468.43 ms
llama_print_timings:      sample time =     257.82 ms /    43 runs   (    6.00 ms per token,   166.79 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    6492.99 ms /    43 runs   (  151.00 ms per token,     6.62 tokens per second)
llama_print_timings:       total time =    6873.88 ms /    44 tokens


{'game_state': 'game on progress', 'active_player': 'Player1', 'message': "Player1 said 'hi', Player2 replied saying 'hello'"}


`pydantic` works great!